# Data preparation

##**Dataset Import from kaggle**

I have uploaded the necessary file to Kaggle as a private dataset, which should enable us to import it more quickly into Google Colab. Let's streamline the import process.

In [1]:
from google.colab import files

In [2]:
# Import the "files" module from the "google.colab" package to easily upload and download files in the Google Colaboratory environment.
from google.colab import files


In [3]:
#Here you just need to upload the API (kaggle.json) file I will be giving you.
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohammadjaberhossain","key":"6869aff67a258c809fca6a89975b115e"}'}

In [4]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!pip install Monai
!pip install matplotlib
!pip install numpy
!pip install tqdm
!pip install glob2
!pip install dicom2nifti
!pip install pytest-shutil
!pip install nibabel
!pip install medpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.8/151.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 29.4 MB/s eta 0:00:00
  Created wheel for medpy: filename=MedPy-0.4.0-py3-none-any.whl size=214945 sha256=2e7ed496323921fb8e2308e3c509b81eaea23dceddd972ff04f8d0d242a1f718
  Stored in directory: /root/.cache/pip/wheels/d4/32/c7/6380ab2edb8cca018d39a0f1d43250fd9791922c963117de46
Successfully built medpy


##**File handling together**

In [6]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content"

In [7]:
# Import the "os" module
import os
# Set the environment variable "KAGGLE_CONFIG_DIR" to the "/content" directory.
os.environ["KAGGLE_CONFIG_DIR"] = "/content"

In [8]:
!kaggle datasets download -d mohammadjaberhossain/finalmicedata

100% 1.72G/1.72G [01:31<00:00, 22.8MB/s]
100% 1.72G/1.72G [01:31<00:00, 20.1MB/s]


In [9]:
#Unzipp the imported zip file
!7z x finalmicedata.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1844136352 bytes (1759 MiB)

Extracting archive: finalmicedata.zip
--
Path = finalmicedata.zip
Type = zip
Physical Size = 1844136352

  0%      0% 3 - data_folder/test_image/20170222CH_Exp3_zfp_M02.nii                                                             0% 5 - data_folder/test_image/20170223CH_Exp3_zf_M15.nii                                                            1% 7 - data_folder/test_image/20170627CH_Exp6_ZFPtm1a_M01.nii

In [10]:
folder_path = '/content/data_folder/train_image'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in Train Image folder:", num_files)

folder_path = '/content/data_folder/train_mask'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in train mask folder:", num_files)

folder_path = '/content/data_folder/test_image'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in test image folder:", num_files)

folder_path = test_mask_path = '/content/data_folder/test_mask'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in test Mask folder:", num_files)

folder_path = '/content/data_folder/val_image'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in val image folder:", num_files)

folder_path = '/content/data_folder/val_mask'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in val mask folder:", num_files)

Number of files in Train Image folder: 231
Number of files in train mask folder: 231
Number of files in test image folder: 66
Number of files in test Mask folder: 66
Number of files in val image folder: 34
Number of files in val mask folder: 34


**Install all the required dependencies.**

In [11]:
import os
from glob import glob
import shutil
from tqdm import tqdm
import dicom2nifti
import numpy as np
import nibabel as nib

In [12]:
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

In [13]:
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    RandRotated,
    RandAffined,
)
from monai.transforms import NormalizeIntensityd
from monai.transforms import NormalizeIntensityd
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism
import os
from glob import glob
import shutil
from tqdm import tqdm
import numpy as np
import nibabel as nib
from monai.transforms import NormalizeIntensityd
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism
from monai.utils import set_determinism
from monai.utils import first
import matplotlib.pyplot as plt
import torch
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss

from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    AsDiscreted,
    EnsureChannelFirstd,
)

def prepare(in_dir, pixdim=(0.0997441, 0.0997441, 0.5), a_min=-200, a_max=200, spatial_size=[256,256,32], cache=False):
    path_train_volumes = sorted(glob(os.path.join(in_dir, "selected_images16", "*.nii")))
    path_train_segmentation = sorted(glob(os.path.join(in_dir, "selected_masks16", "*.nii")))

    path_test_volumes = sorted(glob(os.path.join(in_dir, "test_image", "*.nii")))
    path_test_segmentation = sorted(glob(os.path.join(in_dir, "test_mask", "*.nii")))

    path_val_volumes = sorted(glob(os.path.join(in_dir, "val_image", "*.nii")))
    path_val_segmentation = sorted(glob(os.path.join(in_dir, "val_mask", "*.nii")))



    print("path_train_volumes:", path_train_volumes)
    print("path_train_segmentation:", path_train_segmentation)
    print("path_test_volumes:", path_test_volumes)
    print("path_test_segmentation:", path_test_segmentation)
    print("path_val_volumes:", path_val_volumes)
    print("path_val_segmentation:", path_val_segmentation)

    train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
    test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]
    val_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_val_volumes, path_val_segmentation)]


    train_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            AsDiscreted(keys=["seg"], to_onehot=2),
            #AddChanneld(keys=["vol", "seg"]),
            #NormalizeIntensityd(keys=["vol"], subtrahend=0.3, divisor=0.5),
            NormalizeIntensityd(keys=["vol"]),
            #Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            #Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            #ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            #CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            #Resized(keys=["vol", "seg"], spatial_size=spatial_size),
            ToTensord(keys=["vol", "seg"]),

        ]
    )

    generat_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            AsDiscreted(keys=["seg"], to_onehot=2),
            #AddChanneld(keys=["vol", "seg"]),
            #NormalizeIntensityd(keys=["vol"], subtrahend=0.3, divisor=0.5),
            NormalizeIntensityd(keys=["vol"]),
            #Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            #Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            #ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            #CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            #Resized(keys=["vol", "seg"], spatial_size=spatial_size),

            RandAffined(keys=['vol', 'seg'], prob=0.5, translate_range=10),
            #RandRotated(keys=['vol', 'seg'], prob=0.5, range_x=5.0),
            #RandGaussianNoised(keys='image', prob=0.5),
            ToTensord(keys=["vol", "seg"]),
        ]
    )

    test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            AsDiscreted(keys=["seg"], to_onehot=2),
            #AddChanneld(keys=["vol", "seg"]),
            #NormalizeIntensityd(keys=["vol"], subtrahend=0.3, divisor=0.5),
            NormalizeIntensityd(keys=["vol"]),
            #Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            #Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            #ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max,b_min=0.0, b_max=1.0, clip=True),
            #CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            #Resized(keys=["vol", "seg"], spatial_size=spatial_size),
            ToTensord(keys=["vol", "seg"]),


        ]
    )


    val_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            AsDiscreted(keys=["seg"], to_onehot=2),
            NormalizeIntensityd(keys=["vol"]),
            ToTensord(keys=["vol", "seg"]),
        ]
    )

    if cache:
        #train_ds = CacheDataset(data=train_files, transform=train_transforms,cache_rate=1.0)
        train_ds = CacheDataset(data=train_files, transform=generat_transforms,cache_rate=1.0)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = CacheDataset(data=test_files, transform=test_transforms, cache_rate=1.0)
        test_loader = DataLoader(test_ds, batch_size=1)

        val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0)
        val_loader = DataLoader(val_ds, batch_size=1)

        return train_loader, test_loader, val_loader

    else:
        #train_ds = Dataset(data=train_files, transform=train_transforms)
        train_ds = CacheDataset(data=train_files, transform=generat_transforms,cache_rate=1.0)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = Dataset(data=test_files, transform=test_transforms)
        test_loader = DataLoader(test_ds, batch_size=1)

        val_ds = Dataset(data=val_files, transform=val_transforms)
        val_loader = DataLoader(val_ds, batch_size=1)

        return train_loader, test_loader, val_loader

In [14]:
# utility.py

from monai.utils import first
import matplotlib.pyplot as plt
import numpy as np
from monai.losses import DiceLoss
from tqdm import tqdm
import torch
import os
import numpy as np
from medpy.metric import dc
import nibabel as nib
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from medpy.metric.binary import hd95
from medpy.metric.binary import hd

In [16]:
import torch
print("Cuda_available? ", torch.cuda.is_available())
print("Name: ", torch.cuda.get_device_name(0))
print("Device count: ", torch.cuda.device_count())
print("current device: ", torch.cuda.current_device())
print("Select device: ", torch.cuda.device(0))
print("get device name: ",torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #finally got the solution

Cuda_available?  True
Name:  Tesla T4
Device count:  1
current device:  0
Select device:  <torch.cuda.device object at 0x7af4459a6440>
get device name:  Tesla T4


In [17]:
from torch import nn
from torch.nn import Conv3d, BatchNorm3d, ReLU, Sigmoid
import numpy as np
import torch

class ResNet(nn.Module):
    def __init__(self, in_filters):
        super(ResNet, self).__init__()

        self.seq = nn.Sequential(
                ReLU(),
                BatchNorm3d(in_filters),
                Conv3d(in_filters, in_filters, 3, padding=1),
                ReLU(),
                BatchNorm3d(in_filters),
                Conv3d(in_filters, in_filters, 3, padding=1)
            )

    def forward(self, x):
        return x + self.seq(x)

    def __str__(self):
        return "ResNet"

class Bottleneck(nn.Module):
    def __init__(self, in_filters, out_filters):
        super(Bottleneck, self).__init__()

        self.seq = nn.Sequential(
                ReLU(),
                BatchNorm3d(in_filters),
                Conv3d(in_filters, out_filters, 1)
            )

    def forward(self, x):
        return self.seq(x)

    def __str__(self):
        return "Bottleneck"

In [18]:
import torch
from torch import nn
from torch.nn import Conv3d
from torch.nn.functional import interpolate

class RatLesNetv2(nn.Module):

    def __init__(self, modalities, filters):
        super(RatLesNetv2, self).__init__()

        self.conv1 = Conv3d(modalities, filters, 1)

        self.block1 = ResNet(filters)
        self.mp1 = nn.modules.MaxPool3d(2, ceil_mode=True)

        self.block2 = ResNet(filters)
        self.mp2 = nn.modules.MaxPool3d(2, ceil_mode=True)

        self.block3 = ResNet(filters)
        self.mp3 = nn.modules.MaxPool3d(2, ceil_mode=True)

        self.bottleneck1 = Bottleneck(filters, filters)
        self.block4 = ResNet(filters*2)

        self.bottleneck2 = Bottleneck(filters*2, filters)
        self.block5 = ResNet(filters*2)

        self.bottleneck3 = Bottleneck(filters*2, filters)
        self.block6 = ResNet(filters*2)

        self.bottleneck4 = Bottleneck(filters*2, 2)

    def forward(self, x):
        x = self.conv1(x)
        block1_out = self.block1(x)
        block1_size = block1_out.size()

        x = self.mp1(block1_out)
        block2_out = self.block2(x)
        block2_size = block2_out.size()

        x = self.mp2(block2_out)
        block3_out = self.block3(x)
        block3_size = block3_out.size()

        x = self.mp3(block3_out)
        b1 = self.bottleneck1(x) #upro here

        x = interpolate(b1, block3_size[2:], mode="trilinear")
        x = torch.cat([x, block3_out], dim=1)

        block4_out = self.block4(x)
        b2 = self.bottleneck2(block4_out)

        x = interpolate(b2, block2_size[2:], mode="trilinear")
        x = torch.cat([x, block2_out], dim=1)

        block5_out = self.block5(x)
        b3 = self.bottleneck3(block5_out)

        x = interpolate(b3, block1_size[2:], mode="trilinear")
        x = torch.cat([x, block1_out], dim=1)

        block6_out = self.block6(x)
        b4 = self.bottleneck4(block6_out)

        #softed = torch.functional.F.softmax(b4, dim=1)
        # Must be a tuple
        return b4

In [19]:
filters = 32
modalities = 1

In [20]:
device = torch.device('cuda')
model = RatLesNetv2(modalities, filters).to(device)

In [21]:
device = torch.device('cuda')
model.load_state_dict(torch.load('/content/RatLesNetv2-alldata.model'))  # load the pre-trained weights


<All keys matched successfully>

In [22]:

# Freeze the encoder layers
for param in model.conv1.parameters():
    param.requires_grad = False
for param in model.block1.parameters():
    param.requires_grad = False
for param in model.mp1.parameters():
    param.requires_grad = False
for param in model.block2.parameters():
    param.requires_grad = False
for param in model.mp2.parameters():
    param.requires_grad = False
for param in model.block3.parameters():
    param.requires_grad = False
for param in model.mp3.parameters():
    param.requires_grad = False
for param in model.bottleneck1.parameters():
    param.requires_grad = False

# Enable gradient computation for the decoder layers
for param in model.block4.parameters():
    param.requires_grad = True
for param in model.bottleneck2.parameters():
    param.requires_grad = True
for param in model.block5.parameters():
    param.requires_grad = True
for param in model.bottleneck3.parameters():
    param.requires_grad = True
for param in model.block6.parameters():
    param.requires_grad = True
for param in model.bottleneck4.parameters():
    param.requires_grad = True


In [23]:
model.to(device)

RatLesNetv2(
  (conv1): Conv3d(1, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (block1): ResNet(
    (seq): Sequential(
      (0): ReLU()
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (3): ReLU()
      (4): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    )
  )
  (mp1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (block2): ResNet(
    (seq): Sequential(
      (0): ReLU()
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (3): ReLU()
      (4): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Conv3d(32, 32, kernel_size=(3, 3, 3),

In [24]:
data_dir = '/content/data_folder'

In [25]:
data_in = prepare(data_dir, cache=True)

/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


path_train_volumes: ['/content/data_folder/selected_images16/20170627CH_Exp6_ZFPtm1a_M02.nii', '/content/data_folder/selected_images16/20170712CH_Exp7_ZFPtm1a_M01.nii', '/content/data_folder/selected_images16/20170712CH_Exp7_ZFPtm1a_M02.nii', '/content/data_folder/selected_images16/20170712CH_Exp7_ZFPtm1a_M06.nii', '/content/data_folder/selected_images16/20170712CH_Exp7_ZFPtm1a_M10.nii', '/content/data_folder/selected_images16/20170713CH_Exp7_ZFPtm1a_M24.nii', '/content/data_folder/selected_images16/20170714CH_Exp7_ZFPtm1a_M25.nii', '/content/data_folder/selected_images16/20170921CH_Exp09_M13.nii', '/content/data_folder/selected_images16/20180323CH_Exp3_Phag_M13.nii', '/content/data_folder/selected_images16/20180710CH_Exp8_M03.nii', '/content/data_folder/selected_images16/20180801CH_Exp9_M15.nii', '/content/data_folder/selected_images16/20180802CH_Exp9_M25.nii', '/content/data_folder/selected_images16/20180803CH_Exp9_M33.nii', '/content/data_folder/selected_images16/20190220CH_Exp1_M47

Loading dataset: 100%|██████████| 34/34 [00:07<00:00,  4.55it/s]


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Result Calcualtion with SD


In [28]:
import torch
import os
import numpy as np
from medpy.metric import dc
import nibabel as nib
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from medpy.metric.binary import hd95
from medpy.metric.binary import hd

def MyDiceLoss(y_pred, y_true):
    #from IPython import embed; embed(); asd
    axis = list([i for i in range(2, len(y_true.shape))])
    num = 2 * torch.sum(y_pred * y_true, axis=axis)
    denom = torch.sum(y_pred + y_true, axis=axis)
    d = (1 - torch.mean(num / (denom + 1e-6)))
    return d



loss = MyDiceLoss


dice_list_test1 = []
train_loader, test_loader, val_loader = data_in
save_loss_test1 = []
save_metric_test1 = []
hd95_test1_epoch = []
voxel_spacing = (0.0997441, 0.0997441, 0.5)
count =1
countt= 1
#model.load_state_dict(torch.load('/content/ModelUnetNoAugment/best_metric_model.pth'))
model.load_state_dict(torch.load('/content/drive/MyDrive/Thesis/Final/TL_16images_augment_partial/best_metric_model.pth'))

model = model.to(device)
model.eval()
with torch.no_grad():
  test1_epoch_loss = 0
  test1_metric = 0
  epoch_metric_test1 = 0
  test1_step = 0

  hd95_list_test1 = []

  for test1_data in test_loader:

      test1_step += 1

      test1_volume = test1_data["vol"]
      test1_label = test1_data["seg"]
      test1_label = test1_label != 0
      test1_volume, test1_label = (test1_volume.to(device), test1_label.to(device),)

      test1_outputs = model(test1_volume)

      test1_loss = loss(test1_outputs, test1_label)
      test1_epoch_loss += test1_loss.item()
      #test_metric = dice_metric(test_outputs, test_label)
      lesion_output = torch.softmax(test1_outputs[0], axis=0).detach().cpu().numpy()
      lesion_output = np.argmax(lesion_output, axis=0)
      lesion_label = test1_label[0].detach().cpu().numpy()
      lesion_label = np.argmax(lesion_label, axis=0)
      test1_metric = dc(lesion_output, lesion_label)
      epoch_metric_test1 += test1_metric
      dice_list_test1.append(test1_metric)

      if np.count_nonzero(lesion_output) != 0 and np.count_nonzero(lesion_label) != 0:
          hd95_test1 = hd95(lesion_output, lesion_label, voxelspacing=voxel_spacing, connectivity=1)


          print(f'Test HD95: {hd95_test1}')
          hd95_list_test1.append(hd95_test1)
          print(countt)
          countt = countt + 1
      else:
          print("Skipping HD95 calculation for this batch in testing due to no predicted lesions.")
          print(countt)
          countt = countt + 1

# Compute the average HD95 for this test epoch
test1_hd95_avg = np.mean(hd95_list_test1)
hd95_test1_epoch.append(test1_hd95_avg)
print(f'Test HD95 average: {test1_hd95_avg}')

std_hd95_test1 = np.std(hd95_list_test1)
print(f'Standard deviation of HD95: {std_hd95_test1}')


test1_epoch_loss /= test1_step

print(f'Final test_loss_epoch: {test1_epoch_loss:.4f}')
#save_loss_test1.append(test1_epoch_loss)
#np.save(os.path.join(model_dir, 'loss_test.npy'), save_loss_test1)

epoch_metric_test1 /= test1_step
print(f'Final test_dice_epoch: {epoch_metric_test1:.4f}')

std_dice_test1 = np.std(dice_list_test1)
print(f'Standard deviation of test DICE: {std_dice_test1}')

Test HD95: 0.4254674568549905
1
Test HD95: 0.5098518269897736
2
Test HD95: 0.2992323
3
Test HD95: 0.5
4
Test HD95: 0.4994881999999999
5
Test HD95: 1.2221336611030928
6
Test HD95: 0.5098518269897736
7
Test HD95: 1.3585489561325532
8
Test HD95: 0.5
9
Test HD95: 1.615595289491905
10
Test HD95: 1.7048691571835488
11
Test HD95: 1.802048845995339
12
Test HD95: 0.2230345879545368
13
Test HD95: 0.1994882
14
Test HD95: 0.1994882
15
Test HD95: 0.5525137928983934
16
Test HD95: 0.5085971122657501
17
Test HD95: 0.3596324669750077
18
Test HD95: 0.31541853916360085
19
Test HD95: 0.2230345879545368
20
Test HD95: 0.28211891797339644
21
Test HD95: 0.8111130037653261
22
Test HD95: 0.28211891797339644
23
Test HD95: 0.822510919664341
24
Test HD95: 0.5984646
25
Test HD95: 0.5098518269897736
26
Test HD95: 0.5
27
Test HD95: 0.5
28
Test HD95: 1.0049621313685457
29
Test HD95: 0.5
30
Test HD95: 1.027021660675119
31
Test HD95: 0.5
32
Test HD95: 0.3989764
33
Test HD95: 0.4112554598321705
34
Test HD95: 0.5
35
Test 

## 8*2= 16 images

In [29]:
# Import necessary libraries
import os
import numpy as np
import nibabel as nib
import shutil
np.random.seed(42)

# Define the paths to the image and mask folders
image_folder = "/content/data_folder/train_image"
mask_folder = "/content/data_folder/train_mask"

# Create a list of all mask file names
mask_files = os.listdir(mask_folder)

# Initialize a dictionary to store lesion sizes
lesion_sizes = {}

# Iterate over the mask files
for mask_file in mask_files:
    if mask_file.endswith("_mask.nii"):
        # Extract the corresponding image file name
        image_file = mask_file[:-9] + ".nii"

        # Load the image file and its header
        image_path = os.path.join(image_folder, image_file)
        image_header = nib.load(image_path).header

        # Obtain the voxel size from the image header
        voxel_size = np.prod(image_header.get_zooms())

        # Load the mask file
        mask_path = os.path.join(mask_folder, mask_file)
        mask_data = nib.load(mask_path).get_fdata()

        # Calculate the size of the lesion in millimeters
        lesion_size = np.sum(mask_data) * voxel_size

        # Store the lesion size in the dictionary
        lesion_sizes[image_file] = lesion_size

# Convert the lesion sizes to an array
lesion_sizes_array = np.array(list(lesion_sizes.values()))

# Generate bins for the Gaussian distribution based on lesion sizes
bins = np.linspace(np.min(lesion_sizes_array), np.max(lesion_sizes_array), 9) # create 8 bins
class_labels = np.digitize(lesion_sizes_array, bins)

# Initialize a dictionary to store the selected images
selected_images = {}

# Select 5 images from each class based on the Gaussian distribution
for class_label in range(1, 9):
    class_indices = np.where(class_labels == class_label)[0]
    if len(class_indices) > 0:
        selected_indices = np.random.choice(class_indices, min(2, len(class_indices)), replace=False) # choose 5 or less if not available
        selected_images_from_class = [list(lesion_sizes.keys())[i] for i in selected_indices]
        selected_images.update({img: class_label for img in selected_images_from_class})

# If less than 40 images were selected, select additional images randomly
while len(selected_images) < 16:
    for class_label in range(1, 9):
        class_indices = np.where(class_labels == class_label)[0]
        leftover_indices = [idx for idx in class_indices if list(lesion_sizes.keys())[idx] not in selected_images]
        if len(leftover_indices) > 0:
            selected_index = np.random.choice(leftover_indices)
            selected_image = list(lesion_sizes.keys())[selected_index]
            selected_images[selected_image] = class_label
            if len(selected_images) >= 16:
                break

# Define the paths to the output folders
output_image_folder = "/content/data_folder/selected_images16"
output_mask_folder = "/content/data_folder/selected_masks16"

# Create the output folders if they don't exist
os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_mask_folder, exist_ok=True)

# Copy the selected images and masks to the output folders
for selected_image in selected_images:
    # Copy the image
    source_image_path = os.path.join(image_folder, selected_image)
    destination_image_path = os.path.join(output_image_folder, selected_image)
    shutil.copy2(source_image_path, destination_image_path)

    # Copy the mask
    mask_name = selected_image[:-4] + "_mask.nii"
    source_mask_path = os.path.join(mask_folder, mask_name)
    destination_mask_path = os.path.join(output_mask_folder, mask_name)
    shutil.copy2(source_mask_path, destination_mask_path)

    # Print the information
    print("Image:", selected_image)
    print("Mask:", mask_name)
    print("Class:", selected_images[selected_image])
    print("Lesion Size (mm^3):", lesion_sizes[selected_image])
    print()

print("Selected images and masks have been copied to separate folders.")



Image: 20170921CH_Exp09_M13.nii
Mask: 20170921CH_Exp09_M13_mask.nii
Class: 1
Lesion Size (mm^3): 13.03499849466607

Image: 20180323CH_Exp3_Phag_M13.nii
Mask: 20180323CH_Exp3_Phag_M13_mask.nii
Class: 1
Lesion Size (mm^3): 8.359998255968094

Image: 20190820CH_Exp14_Senp7_M13.nii
Mask: 20190820CH_Exp14_Senp7_M13_mask.nii
Class: 2
Lesion Size (mm^3): 20.71499374927953

Image: 20190320CH_Exp4_M11.nii
Mask: 20190320CH_Exp4_M11_mask.nii
Class: 2
Lesion Size (mm^3): 20.025007012300193

Image: 20180802CH_Exp9_M25.nii
Mask: 20180802CH_Exp9_M25_mask.nii
Class: 3
Lesion Size (mm^3): 47.715021152514964

Image: 20180710CH_Exp8_M03.nii
Mask: 20180710CH_Exp8_M03_mask.nii
Class: 3
Lesion Size (mm^3): 38.66998472996056

Image: 20190220CH_Exp1_M47.nii
Mask: 20190220CH_Exp1_M47_mask.nii
Class: 4
Lesion Size (mm^3): 55.38002455048263

Image: 20180801CH_Exp9_M15.nii
Mask: 20180801CH_Exp9_M15_mask.nii
Class: 4
Lesion Size (mm^3): 67.81003006082028

Image: 20170712CH_Exp7_ZFPtm1a_M02.nii
Mask: 20170712CH_Exp7

In [30]:
folder_path = '/content/data_folder/selected_images16'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in Train Image folder:", num_files)

folder_path = '/content/data_folder/selected_masks16'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in train mask folder:", num_files)



Number of files in Train Image folder: 16
Number of files in train mask folder: 16
